1. API
2. data cleaning
3. data storing
4. data visualization
5. data analysis

In [60]:

import requests
import pandas as pd
import tqdm
import psycopg2
import plotly.graph_objects as go
from plotly.subplots  import make_subplots
import nbformat
import io

## Request data through API 

In [79]:
fieids_url = {
 "artists" : "http://y.saoju.net/yyj/api/artist/",
 "produces" : "http://y.saoju.net/yyj/api/produce/",
"musicals":"http://y.saoju.net/yyj/api/musical/",
 "musicalproduces":"http://y.saoju.net/yyj/api/musicalproduces/",
"citys" : "http://y.saoju.net/yyj/api/city/",
"theatres":"http://y.saoju.net/yyj/api/theatre/", 
"stages":"http://y.saoju.net/yyj/api/stage/",
"shows":"http://y.saoju.net/yyj/api/search_day/?date={}"

}
columns_needed = {
    "citys":["pk","fields.name"],
    "artists": ["pk","fields.name","fields.note"],
    "musicals":["pk",'fields.name', 'fields.is_original',
       'fields.progress', 'fields.premiere_date',
       'fields.info'],
    "produces":['pk', 'fields.name'],
    "musicalproduces":['pk', 'fields.title', 'fields.musical','fields.produce'],
    "theatres":['pk', 'fields.name','fields.city'],
    "stages":['pk', 'fields.name','fields.theatre', 'fields.seats'],
    
}

database_schema = """

    CREATE TABLE artists (
        artist_id SERIAL PRIMARY KEY,
        name VARCHAR(50),
        note VARCHAR(30)
    );
    
    CREATE TABLE citys (
        city_id SERIAL PRIMARY KEY,
        city_name VARCHAR(20)
    );


    CREATE TABLE musicals (
        musical_id SERIAL PRIMARY KEY,
        name VARCHAR(30),
        is_original boolean,
        progress varchar(5),
        premiere_date date,
        info varchar(300)
    );
    CREATE TABLE produces (
        produce_id SERIAL PRIMARY KEY,
        name VARCHAR(100)
    );
    CREATE TABLE musicalproduces (
        musicalproduce_id SERIAL PRIMARY KEY,
        title VARCHAR(20),
        musical_id INT REFERENCES musicals(musical_id),
        produce_id INT REFERENCES produces(produce_id)
        
    );
    CREATE TABLE theatres (
        theatre_id SERIAL PRIMARY KEY,
        name VARCHAR(30),
        city_id INT REFERENCES citys(city_id)
    );    
    CREATE TABLE stages (
        stage_id SERIAL PRIMARY KEY,
        name VARCHAR(30),
        theatre_id INT,
        seats INT
    ); 
    create table shows(
    show_id SERIAL PRIMARY KEY,
    date date,
    city varchar(10),
    musical varchar(50),
    casts varchar(100),
    theatre varchar(50)   
)
"""
location_dict = {
    "上海":"Shanghai",
    "北京":"Beijing",
    "深圳":"Shenzhen",
    "南京":"Nanjing",
    "成都":"Chengdu",
    "杭州":"Hangzhou",
    "广州":"Guangzhou",
    "苏州":"Suzhou",
    "西安":"Xian",
    "武汉":"Wuhan"
}

In [108]:
# speperate shows data for it needs additional parameters and need to be processed
def request_all_data(data_dict:dict):
    for key in fieids_url.keys():
        response = requests.get(fieids_url[key])
        if response.status_code == 200:
            data_dict[key] = pd.json_normalize(response.json())
            print(f"successfully request on {fieids_url[key]}, status code {response.status_code}.")
        else:
            print(f"Error occur when requesting {fieids_url[key]}, status code {response.status_code}.")

def get_shows_by_date(date:str):
    url = fieids_url["shows"].format(date)
    response = requests.get(url)
    if response.status_code == 200:
        print(f"successfully request on {url}, status code {response.status_code}.")
        data = pd.json_normalize(response.json())
        if data["show_list"][0]:  # check if show_list is empty
            shows = pd.json_normalize(data["show_list"][0])
            shows['cast'] = shows['cast'].apply(lambda x: ' '.join([j["artist"] for j in x]))
            shows["date"] = date
            return shows
        else:
            print(f"{date} has no show data.")
            return pd.DataFrame()
    else:
        print(f"Error occur when requesting {url}, status code {response.status_code}.")

def get_shows_by_date_range(start_date:str, end_date:str):
    date_range = pd.date_range(start_date, end_date)
    shows = pd.DataFrame()
    for date in tqdm.tqdm(date_range):
        shows = pd.concat([shows, get_shows_by_date(date.strftime("%Y-%m-%d"))])
    shows['date'] = pd.to_datetime(shows['date'].astype(str) + ' ' + shows['time'])
    shows = shows[["date","city","musical","cast","theatre"]]
    return shows


def save_all2csv(data_dict:dict):
    for key in data_dict.keys():
        data_dict[key].to_csv(f"../original_data/{key}")

In [109]:
data_dict = {}
request_all_data(data_dict)


successfully request on http://y.saoju.net/yyj/api/artist/, status code 200.
successfully request on http://y.saoju.net/yyj/api/produce/, status code 200.
successfully request on http://y.saoju.net/yyj/api/musical/, status code 200.
successfully request on http://y.saoju.net/yyj/api/musicalproduces/, status code 200.
successfully request on http://y.saoju.net/yyj/api/city/, status code 200.
successfully request on http://y.saoju.net/yyj/api/theatre/, status code 200.
successfully request on http://y.saoju.net/yyj/api/stage/, status code 200.
successfully request on http://y.saoju.net/yyj/api/search_day/?date={}, status code 200.


In [85]:

# 2 month shows data needs about 12s, to see all data(about 18 months), load it from csv. csv data also comes from this api.
# I put the csv file in google drive, and use the link to load data. or you can still use the api to get data.
# data_dict["shows"] = get_shows_by_date_range("2024-07-01", "2024-09-01")

url = "https://drive.google.com/uc?export=download&id=1Y0l7vt5gG-9Vqdts1NiKDcTWoGTt_vPj"
response = requests.get(url)
content = response.content
data_dict["shows"] = pd.read_csv(io.StringIO(content.decode('utf-8')),index_col=0)

In [87]:

for key in fieids_url.keys():
    if key == "shows":
        continue
    data_dict[key] = data_dict[key][columns_needed[key]]
    data_dict[key] = data_dict[key].replace(pd.NA," ")
    if key == "musicalproduces":
        new_order = ['pk','fields.title','fields.musical',  'fields.produce']
        data_dict[key] = data_dict[key][new_order]
    elif key == "theatres":
        new_order = ['pk', 'fields.name', 'fields.city']
        data_dict[key] = data_dict[key][new_order]
    elif key == "stages":
        new_order = ['pk',  'fields.name','fields.theatre', 'fields.seats']
        data_dict[key] = data_dict[key][new_order]

 

## To database

In [89]:
conn = psycopg2.connect(
    host ="localhost",
dbname ="a1",
user = "postgres",
password ="is5126",
port=5433)
cur = conn.cursor()


In [90]:

cur.execute(database_schema)

conn.commit()

In [91]:
data_dict["shows"]

,date,city,musical,cast,theatre
0,2023-01-01 11:00:00,成都,熊猫,NaN,熊猫剧院
1,2023-01-01 13:00:00,上海,致爱,王洁璐 赵嘉豪,星空间11号·MOriginal Box
2,2023-01-01 14:00:00,上海,夜半歌声,钟嘉诚 喻越越 陆宇鹏 江珊 丁伟 李泽美 王珏语涵 朱微之 黄湙云 赵雨阳,中国大戏院
3,2023-01-01 14:00:00,上海,月亮与六便士,白一博 暴晓萱 王壹 盛可欣,星空间815号·缪时客小剧场
4,2023-01-01 14:00:00,上海,翻国王棋,丁宇佳 张琦铭 王颢珏 车鸣笛 周波 杜钇樵,星空间77号·鲸剧场
...,...,...,...,...,...
16049,2024-09-01 19:00:00,南京,醉后赢家,NaN,开心麻花方橙市
16050,2024-09-01 15:30:00,西安,丝路之声,谭建业 田恬 刘靖 孙晶俐 张书豪 尉梦飞 杨雨婷 王笑语 林梦圆 殷赫阳,丝路欢乐世界·琴音剧场
16051,2024-09-01 15:00:00,南昌,画中人,张鑫澳 王珂 郭浩然,Lilime泠空间剧场
16052,2024-09-01 19:30:00,南昌,画中人,徐晞城 施天玮 李信,Lilime泠空间剧场


In [92]:

data_dict["shows"]['date'] = pd.to_datetime(data_dict[key]['date']).dt.strftime('%Y-%m-%d %H:%M:%S')


for key in data_dict.keys():
    
    try:
        num_columns = len(data_dict[key].columns)
        placeholders = ', '.join(['%s'] * num_columns)
        # Define the SQL insert query with placeholders for the data
        if key != "shows":
            insert_query = f"INSERT INTO {key} VALUES ({placeholders})"
        else:
            insert_query = f"INSERT INTO shows (date, city, musical, casts, theatre) VALUES ({placeholders})"

    
        # Iterate over DataFrame rows as namedtuples (use index=False to avoid including the index)
        for row in data_dict[key].itertuples(index=False, name=None):
            try:
                cur.execute(insert_query, row)
            except psycopg2.Error as e:
                print(f"An error occurred: {e}")
                conn.rollback()  # Rollback in case of error
    
        # Commit the transaction
        conn.commit()
        print(f"{key} Data inserted successfully")
    
    except psycopg2.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()  # Rollback in case of error
    

Data inserted successfully
Data inserted successfully
Data inserted successfully
Data inserted successfully
Data inserted successfully
Data inserted successfully
Data inserted successfully
Data inserted successfully


## data EDA

In [93]:

def get_table_data(cur:psycopg2.connect,table_names) -> dict :
    df_list = {}
    for name in table_names:

        cur.execute(f"""
            SELECT * FROM {name}
            """)
        results = cur.fetchall()
        columns_name = [desc[0] for desc in cur.description]
        df_list[name] = pd.DataFrame(results,columns = columns_name)
    return df_list


def use_sql_query(cur,sql_query:str,params=None):
    cur.execute(sql_query,params)
    results = cur.fetchall()
    columns_name = [desc[0] for desc in cur.description]
        
    return pd.DataFrame(results,columns = columns_name)


## SQL Query

In [94]:


theatre_number_in_citys = """
    select COUNT(c.city_name) as numberOfTheatre, c.city_name from theatres t
    join citys c on t.city_id = c.city_id
    group by c.city_name
    order by numberOfTheatre desc
    """
seat_number_in_citys = """
select SUM(s.seats) as totalNumberOfSeats,c.city_name from theatres t
join citys c on t.city_id = c.city_id
join stages s on s.theatre_id = t.theatre_id
group by c.city_name
order by totalNumberOfSeats desc
"""
shows_performed_last_month = """
select count(show_id) as shows_perform_a_month, city  from shows
where shows.date >= current_date - interval '30 days'
group by city
order by shows_perform_a_month desc
"""
original_musical_persentage_by_date_range = """
SELECT 
    is_original,
    COUNT(*) as count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentage
FROM musicals
WHERE premiere_date BETWEEN %s AND %s
GROUP BY is_original
ORDER BY count DESC

"""
musical_perform_by_date_range = """
SELECT 
    city,
    COUNT(*) as count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentage
FROM shows
WHERE date BETWEEN %s AND %s
GROUP BY city
ORDER BY count DESC
"""
get_monthly_number_of_shows_by_city = """
SELECT 
    city,
    EXTRACT(YEAR FROM date) AS year,
    EXTRACT(MONTH FROM date) AS month,
    COUNT(show_id) AS total_shows
FROM 
    shows
GROUP BY 
    city, 
    EXTRACT(YEAR FROM date), 
    EXTRACT(MONTH FROM date)
ORDER BY 
    city, year, month;

"""
musical_list_by_date_range = """
SELECT 
*
FROM shows
WHERE date BETWEEN %s AND %s
"""

In [95]:
table_names = ["shows","theatres","citys","musicals","produces"]
datas = get_table_data(cur=cur,table_names=table_names)


In [8]:
datas["musicals"].head(10)

,musical_id,name,is_original,progress,premiere_date,info
0,1,I LOVE YOU…,False,SS,2006-12-09,"百老汇音乐剧中文版\r\n全名《I Love You, You’re Perfect, No..."
1,2,空中花园谋杀案,True,SS,2009-05-26,
2,3,我，堂吉诃德,False,SS,2015-12-18,百老汇经典音乐剧中文版\r\n原版作者：\r\n剧本：戴尔·沃瑟曼 Dale Wasserm...
3,4,我的遗愿清单,False,SS,2017-08-08,原版韩国\r\n制作人：姜秉沅\r\n制作出品：LIVE Corp.\r\n作曲：金潓成
4,5,我变了我没变,True,SS,2018-01-06,
5,6,近乎正常,False,SS,2018-08-03,原版美国\r\n作曲 汤姆·基特 TOM KITT\r\n编剧及作词 布赖恩·约克 BRIA...
6,7,白夜行,True,SS,2018-11-30,官方授权：株式会社集英社\r\n原作：东野圭吾
7,8,袁隆平,True,SS,2017-06-18,
8,9,献给阿尔吉侬的花束,False,SS,2019-12-11,原版韩国\r\n原版剧本/导演 李泫圭\r\n作曲/音乐总监 张少瑛
9,10,FLAMES火焰,False,SS,2019-12-24,原版英国，悬疑音乐剧


In [98]:
datas["musicals"]["info"] = datas["musicals"]["info"].apply(lambda x: "UNKNOW" if x == " " or pd.isna(x) else x)


In [99]:
datas["musicals"].isnull().sum()

musical_id       0
name             0
is_original      0
progress         0
premiere_date    0
info             0
dtype: int64

In [100]:
datas["shows"].head(10)

,show_id,date,city,musical,casts,theatre
0,1,2023-01-01,成都,熊猫,NaN,熊猫剧院
1,2,2023-01-01,上海,致爱,王洁璐 赵嘉豪,星空间11号·MOriginal Box
2,3,2023-01-01,上海,夜半歌声,钟嘉诚 喻越越 陆宇鹏 江珊 丁伟 李泽美 王珏语涵 朱微之 黄湙云 赵雨阳,中国大戏院
3,4,2023-01-01,上海,月亮与六便士,白一博 暴晓萱 王壹 盛可欣,星空间815号·缪时客小剧场
4,5,2023-01-01,上海,翻国王棋,丁宇佳 张琦铭 王颢珏 车鸣笛 周波 杜钇樵,星空间77号·鲸剧场
5,6,2023-01-01,上海,莎翁乐园,祝颂皓 张智涵 徐泽辉 刘浩冉,上剧场
6,7,2023-01-01,成都,熊猫,NaN,熊猫剧院
7,8,2023-01-01,上海,卡拉马佐夫兄弟,付翔 王瀚宇 刘令飞 郭虹旭 张泽,上海大剧院别克中剧场
8,9,2023-01-01,上海,阴天,钟舜傲 郭耀嵘 王培杰,上海共舞台
9,10,2023-01-01,上海,沉默的真相,顾易 姜彬 江南 刘乙萱 张艺宝 吴俊鹏 翟松 郝晓辉 刘政 陈姝月 赵洪博,人民大舞台


In [101]:
datas["shows"].isnull().sum()

show_id    0
date       0
city       0
musical    0
casts      0
theatre    0
dtype: int64

## Data Visualization

In [102]:
# get data
theatre_data_df = use_sql_query(cur=cur,sql_query=theatre_number_in_citys).head(10)
musical_perform_df = use_sql_query(cur=cur,sql_query=musical_perform_by_date_range,params=("2023-01-01","2024-12-31"))

#join 2 df
fig_data1 = pd.merge(theatre_data_df,musical_perform_df,left_on = "city_name",right_on = "city",how="left")
fig_data1["city_en"] = fig_data1["city_name"].map(location_dict)

# plot
fig = go.Figure()
fig.add_trace(go.Bar(
    x=fig_data1['city_en'],
    y=fig_data1['numberoftheatre'],
    marker=dict(color="#466b82"),
    name="Number of Theatres"
))
fig.add_trace(go.Scatter(
    x=fig_data1['city_en'],
    y=fig_data1['count'],
    mode='lines+markers',
    line=dict(color='#E91E63', width=2),
    marker=dict(size=8),
    yaxis='y2',
    name="Number of Performance"
))
fig.update_layout(
    yaxis2=dict(
        title='Number of Performances',
        overlaying='y',
        side='right'
    ),
    yaxis=dict(title='Number of Theatres'),
    xaxis=dict(title='City name'),
    legend=dict(x=1.1, y=1),
        autosize=True,
)
fig.show()

In [103]:
# get data
musicals = get_table_data(cur=cur,table_names= ["musicals"] )["musicals"]

musicals["premiere_date"] = pd.to_datetime(musicals["premiere_date"])
musicals["year"] = musicals["premiere_date"].dt.year
musicals = musicals[musicals["year"] > 2017]

year_musical_df = pd.DataFrame(musicals.groupby("year").agg({
    'musical_id': 'count',
    'is_original': 'sum',
}))
year_musical_df["imitation_musical_number"] = year_musical_df["musical_id"] - year_musical_df["is_original"]
year_musical_df["original_musical_ratio"] = year_musical_df["is_original"] / year_musical_df["musical_id"]
year_musical_df.reset_index(inplace=True)

fig_musical_originality = go.Figure()

fig_musical_originality.add_trace(go.Bar(
    x=year_musical_df["year"],
    y=year_musical_df["musical_id"],
    name="Total Musical",
    marker=dict(color="#E91E63"),
    yaxis="y2"

))
fig_musical_originality.add_trace(go.Scatter(
    x=year_musical_df["year"],
    y=year_musical_df["original_musical_ratio"],
    name="original ratio",
    mode='lines+markers',
    marker=dict(color="#466b82"),
    yaxis="y"
))

fig_musical_originality.update_layout(
    title="Musical Originality by Year",
    yaxis=dict(title="Original Ratio",side="right",overlaying="y2",range=[0,1]),
    yaxis2=dict(title="Total Musical",side="left"),
    legend=dict(x=1.1, y=1),
    autosize=True,
    xaxis=dict(title="Year"),
)

fig_musical_originality.show()

In [104]:
musical_2022 = musicals[(musicals["year"] == 2022) & (musicals["is_original"] == 0)]
musical_2023 = musicals[(musicals["year"] == 2023) & (musicals["is_original"] == 0)]
musical_2024 = musicals[(musicals["year"] == 2024) & (musicals["is_original"] == 0)]
county_name = {
    "Broadway":["百老汇"],
    "American":["美国"],
    "Korea":["韩国","韩","韩方"],
    "Thailand":["泰国"],
    "Russia":["俄罗斯"],
    "France":["法国"],
    "Japan":["日本"],
    "Other":["其他"]
}

for key, value in county_name.items():
    musical_2022["info_country"] = musical_2022["info"].apply(lambda x: key if any(v in x for v in value) else musical_2022.loc[musical_2022["info"] == x, "info_country"].iloc[0] if "info_country" in musical_2022.columns else x)
    musical_2023["info_country"] = musical_2023["info"].apply(lambda x: key if any(v in x for v in value) else musical_2023.loc[musical_2023["info"] == x, "info_country"].iloc[0] if "info_country" in musical_2023.columns else x)
    musical_2024["info_country"] = musical_2024["info"].apply(lambda x: key if any(v in x for v in value) else musical_2024.loc[musical_2024["info"] == x, "info_country"].iloc[0] if "info_country" in musical_2024.columns else x)
musical_2022["info_country"] = musical_2022["info_country"].apply(lambda x: x if x in county_name.keys() else "Other")
musical_2023["info_country"] = musical_2023["info_country"].apply(lambda x: x if x in county_name.keys() else "Other")
musical_2024["info_country"] = musical_2024["info_country"].apply(lambda x: x if x in county_name.keys() else "Other")


/var/folders/9s/l93m55bx5w9brq6p8klv2cgr0000gn/T/ipykernel_63009/2151088408.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/9s/l93m55bx5w9brq6p8klv2cgr0000gn/T/ipykernel_63009/2151088408.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/9s/l93m55bx5w9brq6p8klv2cgr0000gn/T/ipykernel_63009/2151088408.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [105]:
fig_musical_imitation_country = make_subplots(rows=1, cols=3, subplot_titles=["2022", "2023", "2024"], specs=[[{'type':'pie'}, {'type':'pie'}, {'type':'pie'}]])
#change color, do not use likely color  
colors = ['#FFA07A', '#6B5B95', '#88B04B', '#F7CAC9', '#92A8D1', '#FFF2E0',"#FFD6AA"]

fig_musical_imitation_country.add_trace(go.Pie(
    labels=musical_2022["info_country"].value_counts().index,
    values=musical_2022["info_country"].value_counts().values,
    name="2022",
    marker=dict(colors=colors)
), row=1, col=1)

fig_musical_imitation_country.add_trace(go.Pie(
    labels=musical_2023["info_country"].value_counts().index,
    values=musical_2023["info_country"].value_counts().values,
    name="2023",
    marker=dict(colors=colors)
), row=1, col=2)

fig_musical_imitation_country.add_trace(go.Pie(
    labels=musical_2024["info_country"].value_counts().index,
    values=musical_2024["info_country"].value_counts().values,
    name="2024",
    marker=dict(colors=colors)
), row=1, col=3)

fig_musical_imitation_country.update_layout(

    title="Imitation Musical by Country",
    legend=dict(x=1.1, y=1),
    autosize=True
)

fig_musical_imitation_country.show()

In [106]:
monthly_shows = use_sql_query(cur=cur,sql_query=get_monthly_number_of_shows_by_city)
monthly_shows["datetime"] = pd.to_datetime(monthly_shows["year"].astype(str) + "-" + monthly_shows["month"].astype(str), format='%Y-%m')
monthly_shows = monthly_shows[(monthly_shows["datetime"] < "2024-09-01")]


shanghai = monthly_shows[monthly_shows["city"] == "上海"]
shanghai_2023 = shanghai.sort_values(by=['datetime'])
shanghai["monthly_ratio"] = (shanghai["total_shows"] - shanghai["total_shows"].shift(1))*100 / shanghai["total_shows"].shift(1)



beijing = monthly_shows[monthly_shows["city"] == "深圳"]
beijing_2023 = beijing.sort_values(by=['datetime'])
beijing["monthly_ratio"] =( beijing["total_shows"] - beijing["total_shows"].shift(1))*100 / beijing["total_shows"].shift(1)


fig3 = go.Figure()
fig3.add_trace(go.Bar(
    x=shanghai["datetime"],
    y=shanghai["total_shows"],
    name="Shanghai",
    yaxis="y"
))
fig3.add_trace(go.Bar(
    x=beijing["datetime"],
    y=beijing["total_shows"],
    name="Shenzhen",
    yaxis="y"
))


fig3.add_trace(go.Scatter(
    x=shanghai["datetime"],
    y=shanghai["monthly_ratio"],
    mode='lines+markers',
    name="Shanghai Ratio",
    yaxis="y2"
))
fig3.add_trace(go.Scatter(
    x=beijing["datetime"],
    y=beijing["monthly_ratio"],
    mode='lines+markers',
    name="Shenzhen Ratio",
    yaxis="y2"
))


fig3.update_layout(
    title="Monthly Distribution of Performances in Different Cities (2023-01-01 --- 2024-09-01)",
    xaxis_title="Month",
    yaxis_title="Total Shows",
    yaxis2=dict(title="Monthly Ratio", overlaying="y", side="right",range=[-500,800]),
    legend_title="City",
    barmode="group"
)

fig3.show()

/var/folders/9s/l93m55bx5w9brq6p8klv2cgr0000gn/T/ipykernel_63009/2414409729.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/9s/l93m55bx5w9brq6p8klv2cgr0000gn/T/ipykernel_63009/2414409729.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [107]:

cur.close()
conn.close()